In [3]:
import mitsuba as mi
import drjit as dr
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import random 
import imageio
import cv2

In [4]:
scene_path = os.path.join('', 'scenes') 
results_spatio_path = os.path.join('','results_spatio') 
mi.set_variant('scalar_rgb')
texture_path = os.path.join(scene_path,'textures')

tokki = os.path.join(scene_path,'meshes/bunny.ply')


In [7]:
import cgi, os
files = os.listdir('dir/graycode_pattern/')

for f in files:
    s = "dir/graycode_pattern/"
    s = s + f
    scene1 = mi.load_dict({
        'type':'scene',
        'integrator': {'type': 'direct'},
        'light': {
            'type': 'projector',
            'irradiance': {
                'type': 'bitmap',
                'filename': s,
            },
            'fov': 45,
            'to_world': mi.ScalarTransform4f.look_at(
                origin=[0, 0, -0.9],
                target=[0, 1, -15],
                up=[0, 0, 1]
            )
            },
        'tok':{
            'type': 'ply',
            'filename': tokki,
            'to_world': mi.ScalarTransform4f.translate([0, 0, -1.3]),
            'bsdf': {
                'type': 'diffuse',
                'reflectance': {'type': 'rgb', 'value': [0.1, 0.2, 0.3]},
            }
        }
    })
    cam1 = mi.load_dict({
        'type': 'perspective',
        'fov': 45, #24.6 mm
        'to_world': mi.ScalarTransform4f.look_at(
            origin=[0, 0, -1.1],
            target=[0, 1, -15],
            up=[0, 0, 1]
        ),
        'film2': {
            'type': 'hdrfilm',
            'width': 360,
            'height': 360,
        }
        # 'distortion': [0, 0, 0, 0, 0]
    })
    image = mi.render(scene=scene1, sensor=cam1, spp=64)
    image = np.clip(image * 255, 0, 255).astype(np.uint8)
    st = "./results/result_" + f
    # Save the image as a PNG file using imageio
    imageio.imwrite(st, image)
